# Tables with results from alternative computational setups
**Author**: Egor Trushin  

In [1]:
import os
import numpy as np

In [2]:
def parse_en(filename, keyword):
    """Parse energy from SCEXX/USCEXX/KSINV/UKSINV output using keyword."""
    energy = None
    for line in open(filename, "r", encoding="utf-8"):
        if "NOT" in line:
            energy = None
        if keyword in line:
            energy = float(line.split()[-1])
    return energy

In [3]:
def parse_exx_gaps(outfile):
    """Parse HOMO-LUMO gaps from EXX output."""
    gap_alpha = None
    gap_beta = None
    l_problem = False
    for line in open(outfile):
        if "LUMO-HOMO (alpha):" in line:
            gap_alpha = float(line.split()[-1])
        if "LUMO-HOMO (beta):" in line:
            gap_beta = float(line.split()[-1])
        if "SCF NOT" in line:
            l_problem = True
    if l_problem:
        gap_alpha = None
        gap_beta = None
    return gap_alpha, gap_beta

In [4]:
def parse_ksinv_gaps(outfile):
    """Parse HOMO-LUMO gaps from KSINV output."""
    gap_alpha = None
    gap_beta = None
    l_problem = False
    for line in open(outfile):
        if "LUMO-HOMO:" in line:
            gap_alpha = float(line.split()[-2])
            gap_beta = float(line.split()[-1])
        if "SCF NOT" in line:
            l_problem = True
    if l_problem:
        gap_alpha = None
        gap_beta = None
    return gap_alpha, gap_beta

## aug-cc-pwCVQZ orbital basis set (CAHF)

In [5]:
DATA_PATH = "Q/RAW_OUTPUTS/EXX_SpinSym_plot"
SYSTEMS = ["B", "C", "N", "O", "F", "Al", "Si", "P", "S", "Cl"]

# Collect HOMO-LUMO gaps
exx_gaps = {}
for system in SYSTEMS:
    outfile = os.path.join(DATA_PATH, system, "output")
    a, b = parse_exx_gaps(outfile)
    exx_gaps[system] = {"a": a, "b": b}
    
# Collect EXX total energies
exx_energies = {}
for system in SYSTEMS:
    outfile = os.path.join(DATA_PATH, system, "output")
    exx_energies[system] = parse_en(outfile, "USCEXX Total energy")

In [6]:
DATA_PATH = "Q/RAW_OUTPUTS/KSINV_CAHF"
SYSTEMS = ["B", "C", "N", "O", "F", "Al", "Si", "P", "S", "Cl"]
REFERENCE = ["RS2", "CISD", "AQCC"]

# Collect HOMO-LUMO gaps
ksinv_cahf_gaps = {}
for system in SYSTEMS:
    ksinv_cahf_gaps[system] = {}
    for ref in REFERENCE:
        outfile = os.path.join(DATA_PATH, ref, system, "output")
        a, b = parse_ksinv_gaps(outfile)
        ksinv_cahf_gaps[system][ref] = {"a": a, "b": b}

# Parse reference energies
ksinv_cahf_ref_en = {}
for system in SYSTEMS:
    ksinv_cahf_ref_en[system] = {}
    for ref in REFERENCE:
        ksinv_cahf_ref_en[system][ref] = {}
        outfile = os.path.join(DATA_PATH, ref, system, "output")
        if ref == "RS2":
            ksinv_cahf_ref_en[system][ref] = parse_en(outfile, "RSPT2 STATE 1.1 Energy")
        elif ref == "CISD":
            ksinv_cahf_ref_en[system][ref] = parse_en(outfile, "MRCI STATE 1.1 Energy")
        elif ref == "AQCC":
            ksinv_cahf_ref_en[system][ref] = parse_en(outfile, "AQCC STATE 1.1 Energy")

In [7]:
print(f'{" "*14} B {" "*18} C {" "*19} O {" "*18} F')
for ref in ["EXX", "RS2", "CISD", "AQCC"]:
    if ref == "EXX":
        B_gap = exx_gaps["B"]["a"]
        B = f'{B_gap:8.5f} {exx_energies["B"]:8.5f}'
        C_gap = exx_gaps["C"]["a"]
        C = f'{C_gap:8.5f} {exx_energies["C"]:8.5f}'
        O_gap = exx_gaps["O"]["b"]
        O = f'{O_gap:8.5f} {exx_energies["O"]:8.5f}'
        F_gap = exx_gaps["F"]["b"]
        F = f'{F_gap:8.5f} {exx_energies["F"]:8.5f}'
    else:
        B_gap = ksinv_cahf_gaps["B"][ref]["a"]
        if B_gap is not None:
            B = f'{B_gap:8.5f} {ksinv_cahf_ref_en["B"][ref]:8.5f}'
        else:
            B = ""
        C_gap = ksinv_cahf_gaps["C"][ref]["a"]
        if C_gap is not None:
            C = f'{C_gap:8.5f} {ksinv_cahf_ref_en["C"][ref]:8.5f}'
        else:
            C = ""
        O_gap = ksinv_cahf_gaps["O"][ref]["b"]
        O = f'{O_gap:8.5f} {ksinv_cahf_ref_en["O"][ref]:8.5f}'
        F_gap = ksinv_cahf_gaps["F"][ref]["b"]
        F = f'{F_gap:8.5f} {ksinv_cahf_ref_en["F"][ref]:8.5f}'
    print(f"{ref:6} {B:20} {C:20} {O:20} {F:20}")

print(f'\n{" "*14} Al {" "*18} Si {" "*19} S {" "*18} Cl')
for ref in ["EXX", "RS2", "CISD", "AQCC"]:
    if ref == "EXX":
        Al_gap = exx_gaps["Al"]["a"]
        Al = f'{Al_gap:8.5f} {exx_energies["Al"]:8.5f}'
        Si_gap = exx_gaps["Si"]["a"]
        Si = f'{Si_gap:8.5f} {exx_energies["Si"]:8.5f}'
        S_gap = exx_gaps["S"]["b"]
        S = f'{S_gap:8.5f} {exx_energies["S"]:8.5f}'
        Cl_gap = exx_gaps["Cl"]["b"]
        Cl = f'{Cl_gap:8.5f} {exx_energies["Cl"]:8.5f}'
    else:
        Al_gap = ksinv_cahf_gaps["Al"][ref]["a"]
        if Al_gap is not None:
            Al = f'{Al_gap:8.5f} {ksinv_cahf_ref_en["Al"][ref]:8.5f}'
        else:
            Al = ""
        Si_gap = ksinv_cahf_gaps["Si"][ref]["a"]
        if Si_gap is not None:
            Si = f'{Si_gap:8.5f} {ksinv_cahf_ref_en["Si"][ref]:8.5f}'
        else:
            Si = ""
        S_gap = ksinv_cahf_gaps["S"][ref]["b"]
        if S_gap is not None:
            S = f'{S_gap:8.5f} {ksinv_cahf_ref_en["S"][ref]:8.5f}'
        else:
            S = ""
        Cl_gap = ksinv_cahf_gaps["Cl"][ref]["b"]
        if Cl_gap is not None:
            Cl = f'{Cl_gap:8.5f} {ksinv_cahf_ref_en["Cl"][ref]:8.5f}'
        else:
            Cl = ""
    print(f"{ref:6} {Al:20} {Si:20} {S:20} {Cl:20}")

               B                    C                     O                    F
EXX     0.00630 -24.52747    0.00643 -37.68581   -0.02790 -74.80758   -0.02846 -99.40645  
RS2    -0.01833 -24.64314                        -0.02646 -75.04654   -0.02827 -99.70769  
CISD   -0.01679 -24.64845   -0.01809 -37.83364   -0.03166 -75.04530   -0.03483 -99.70432  
AQCC   -0.01695 -24.64928   -0.01867 -37.83578   -0.03565 -75.05064   -0.03900 -99.71106  

               Al                    Si                     S                    Cl
EXX     0.00144 -241.87182   0.00143 -288.84996  -0.01368 -397.49975  -0.01260 -459.47656 
RS2    -0.00783 -242.27457  -0.00150 -289.28150  -0.01522 -398.02663  -0.01526 -460.05601 
CISD   -0.00773 -242.26654  -0.00554 -289.26859  -0.01517 -398.00422  -0.01717 -460.03387 
AQCC   -0.00849 -242.27833  -0.00806 -289.28536  -0.02219 -398.03063  -0.02496 -460.06454 


## aug-cc-pwCVTZ orbital basis set (CAHF)

In [8]:
DATA_PATH = "T/RAW_OUTPUTS/EXX_SpinSym_plot"
SYSTEMS = ["B", "C", "N", "O", "F", "Al", "Si", "P", "S", "Cl"]

# Collect HOMO-LUMO gaps
exx_gaps = {}
for system in SYSTEMS:
    outfile = os.path.join(DATA_PATH, system, "output")
    a, b = parse_exx_gaps(outfile)
    exx_gaps[system] = {"a": a, "b": b}
    
# Collect EXX total energies
exx_energies = {}
for system in SYSTEMS:
    outfile = os.path.join(DATA_PATH, system, "output")
    exx_energies[system] = parse_en(outfile, "USCEXX Total energy")

In [9]:
DATA_PATH = "T/RAW_OUTPUTS/KSINV_CAHF"
SYSTEMS = ["B", "C", "N", "O", "F", "Al", "Si", "P", "S", "Cl"]
REFERENCE = ["RS2", "CISD", "AQCC"]

# Collect HOMO-LUMO gaps
ksinv_cahf_gaps = {}
for system in SYSTEMS:
    ksinv_cahf_gaps[system] = {}
    for ref in REFERENCE:
        outfile = os.path.join(DATA_PATH, ref, system, "output")
        a, b = parse_ksinv_gaps(outfile)
        ksinv_cahf_gaps[system][ref] = {"a": a, "b": b}

# Parse reference energies
ksinv_cahf_ref_en = {}
for system in SYSTEMS:
    ksinv_cahf_ref_en[system] = {}
    for ref in REFERENCE:
        ksinv_cahf_ref_en[system][ref] = {}
        outfile = os.path.join(DATA_PATH, ref, system, "output")
        if ref == "RS2":
            ksinv_cahf_ref_en[system][ref] = parse_en(outfile, "RSPT2 STATE 1.1 Energy")
        elif ref == "CISD":
            ksinv_cahf_ref_en[system][ref] = parse_en(outfile, "MRCI STATE 1.1 Energy")
        elif ref == "AQCC":
            ksinv_cahf_ref_en[system][ref] = parse_en(outfile, "AQCC STATE 1.1 Energy")

In [10]:
print(f'{" "*14} B {" "*18} C {" "*19} O {" "*18} F')
for ref in ["EXX", "RS2", "CISD", "AQCC"]:
    if ref == "EXX":
        B_gap = exx_gaps["B"]["a"]
        B = f'{B_gap:8.5f} {exx_energies["B"]:8.5f}'
        C_gap = exx_gaps["C"]["a"]
        C = f'{C_gap:8.5f} {exx_energies["C"]:8.5f}'
        O_gap = exx_gaps["O"]["b"]
        O = f'{O_gap:8.5f} {exx_energies["O"]:8.5f}'
        F_gap = exx_gaps["F"]["b"]
        F = f'{F_gap:8.5f} {exx_energies["F"]:8.5f}'
    else:
        B_gap = ksinv_cahf_gaps["B"][ref]["a"]
        if B_gap is not None:
            B = f'{B_gap:8.5f} {ksinv_cahf_ref_en["B"][ref]:8.5f}'
        else:
            B = ""
        C_gap = ksinv_cahf_gaps["C"][ref]["a"]
        if C_gap is not None:
            C = f'{C_gap:8.5f} {ksinv_cahf_ref_en["C"][ref]:8.5f}'
        else:
            C = ""
        O_gap = ksinv_cahf_gaps["O"][ref]["b"]
        O = f'{O_gap:8.5f} {ksinv_cahf_ref_en["O"][ref]:8.5f}'
        F_gap = ksinv_cahf_gaps["F"][ref]["b"]
        F = f'{F_gap:8.5f} {ksinv_cahf_ref_en["F"][ref]:8.5f}'
    print(f"{ref:6} {B:20} {C:20} {O:20} {F:20}")

print(f'\n{" "*14} Al {" "*18} Si {" "*19} S {" "*18} Cl')
for ref in ["EXX", "RS2", "CISD", "AQCC"]:
    if ref == "EXX":
        Al_gap = exx_gaps["Al"]["a"]
        Al = f'{Al_gap:8.5f} {exx_energies["Al"]:8.5f}'
        Si_gap = exx_gaps["Si"]["a"]
        Si = f'{Si_gap:8.5f} {exx_energies["Si"]:8.5f}'
        S_gap = exx_gaps["S"]["b"]
        S = f'{S_gap:8.5f} {exx_energies["S"]:8.5f}'
        Cl_gap = exx_gaps["Cl"]["b"]
        Cl = f'{Cl_gap:8.5f} {exx_energies["Cl"]:8.5f}'
    else:
        Al_gap = ksinv_cahf_gaps["Al"][ref]["a"]
        if Al_gap is not None:
            Al = f'{Al_gap:8.5f} {ksinv_cahf_ref_en["Al"][ref]:8.5f}'
        else:
            Al = ""
        Si_gap = ksinv_cahf_gaps["Si"][ref]["a"]
        if Si_gap is not None:
            Si = f'{Si_gap:8.5f} {ksinv_cahf_ref_en["Si"][ref]:8.5f}'
        else:
            Si = ""
        S_gap = ksinv_cahf_gaps["S"][ref]["b"]
        if S_gap is not None:
            S = f'{S_gap:8.5f} {ksinv_cahf_ref_en["S"][ref]:8.5f}'
        else:
            S = ""
        Cl_gap = ksinv_cahf_gaps["Cl"][ref]["b"]
        if Cl_gap is not None:
            Cl = f'{Cl_gap:8.5f} {ksinv_cahf_ref_en["Cl"][ref]:8.5f}'
        else:
            Cl = ""
    print(f"{ref:6} {Al:20} {Si:20} {S:20} {Cl:20}")

               B                    C                     O                    F
EXX     0.01339 -24.52360    0.00699 -37.68471   -0.03272 -74.80273   -0.03223 -99.39907  
RS2    -0.00795 -24.63516                        -0.03033 -75.02509   -0.03143 -99.67842  
CISD   -0.00724 -24.64175   -0.01944 -37.82478   -0.03688 -75.02623   -0.03909 -99.67731  
AQCC   -0.00734 -24.64253   -0.01999 -37.82684   -0.04146 -75.03127   -0.04357 -99.68359  

               Al                    Si                     S                    Cl
EXX     0.00563 -241.87112   0.00748 -288.84528  -0.01552 -397.49721  -0.01455 -459.47422 
RS2    -0.00833 -242.20257   0.00496 -289.20695  -0.01680 -397.93915  -0.01711 -459.96307 
CISD   -0.00729 -242.19727  -0.00099 -289.19833  -0.01819 -397.92274  -0.02042 -459.94710 
AQCC   -0.00821 -242.20737  -0.00331 -289.21295  -0.02506 -397.94563  -0.02774 -459.97340 


## aug-cc-pwCV5Z orbital basis set, aug-cc-pVTZ/aug-cc-pVQZ auxiliary basis sets (CAHF)

In [11]:
DATA_PATH = "OEP_Plus/RAW_OUTPUTS/EXX_SpinSym_plot"
SYSTEMS = ["B", "C", "N", "O", "F", "Al", "Si", "P", "S", "Cl"]

# Collect HOMO-LUMO gaps
exx_gaps = {}
for system in SYSTEMS:
    outfile = os.path.join(DATA_PATH, system, "output")
    a, b = parse_exx_gaps(outfile)
    exx_gaps[system] = {"a": a, "b": b}
    
# Collect EXX total energies
exx_energies = {}
for system in SYSTEMS:
    outfile = os.path.join(DATA_PATH, system, "output")
    exx_energies[system] = parse_en(outfile, "USCEXX Total energy")

In [12]:
DATA_PATH = "OEP_Plus/RAW_OUTPUTS/KSINV_CAHF"
SYSTEMS = ["B", "C", "N", "O", "F", "Al", "Si", "P", "S", "Cl"]
REFERENCE = ["RS2", "CISD", "AQCC"]

# Collect HOMO-LUMO gaps
ksinv_cahf_gaps = {}
for system in SYSTEMS:
    ksinv_cahf_gaps[system] = {}
    for ref in REFERENCE:
        outfile = os.path.join(DATA_PATH, ref, system, "output")
        a, b = parse_ksinv_gaps(outfile)
        ksinv_cahf_gaps[system][ref] = {"a": a, "b": b}

# Parse reference energies
ksinv_cahf_ref_en = {}
for system in SYSTEMS:
    ksinv_cahf_ref_en[system] = {}
    for ref in REFERENCE:
        ksinv_cahf_ref_en[system][ref] = {}
        outfile = os.path.join(DATA_PATH, ref, system, "output")
        if ref == "RS2":
            ksinv_cahf_ref_en[system][ref] = parse_en(outfile, "RSPT2 STATE 1.1 Energy")
        elif ref == "CISD":
            ksinv_cahf_ref_en[system][ref] = parse_en(outfile, "MRCI STATE 1.1 Energy")
        elif ref == "AQCC":
            ksinv_cahf_ref_en[system][ref] = parse_en(outfile, "AQCC STATE 1.1 Energy")

In [13]:
print(f'{" "*14} B {" "*18} C {" "*19} O {" "*18} F')
for ref in ["EXX", "RS2", "CISD", "AQCC"]:
    if ref == "EXX":
        B_gap = exx_gaps["B"]["a"]
        B = f'{B_gap:8.5f} {exx_energies["B"]:8.5f}'
        C_gap = exx_gaps["C"]["a"]
        C = f'{C_gap:8.5f} {exx_energies["C"]:8.5f}'
        O_gap = exx_gaps["O"]["b"]
        O = f'{O_gap:8.5f} {exx_energies["O"]:8.5f}'
        F_gap = exx_gaps["F"]["b"]
        F = f'{F_gap:8.5f} {exx_energies["F"]:8.5f}'
    else:
        B_gap = ksinv_cahf_gaps["B"][ref]["a"]
        if B_gap is not None:
            B = f'{B_gap:8.5f} {ksinv_cahf_ref_en["B"][ref]:8.5f}'
        else:
            B = ""
        C_gap = ksinv_cahf_gaps["C"][ref]["a"]
        if C_gap is not None:
            C = f'{C_gap:8.5f} {ksinv_cahf_ref_en["C"][ref]:8.5f}'
        else:
            C = ""
        O_gap = ksinv_cahf_gaps["O"][ref]["b"]
        O = f'{O_gap:8.5f} {ksinv_cahf_ref_en["O"][ref]:8.5f}'
        F_gap = ksinv_cahf_gaps["F"][ref]["b"]
        F = f'{F_gap:8.5f} {ksinv_cahf_ref_en["F"][ref]:8.5f}'
    print(f"{ref:6} {B:20} {C:20} {O:20} {F:20}")

print(f'\n{" "*14} Al {" "*18} Si {" "*19} S {" "*18} Cl')
for ref in ["EXX", "RS2", "CISD", "AQCC"]:
    if ref == "EXX":
        Al_gap = exx_gaps["Al"]["a"]
        Al = f'{Al_gap:8.5f} {exx_energies["Al"]:8.5f}'
        Si_gap = exx_gaps["Si"]["a"]
        Si = f'{Si_gap:8.5f} {exx_energies["Si"]:8.5f}'
        S_gap = exx_gaps["S"]["b"]
        S = f'{S_gap:8.5f} {exx_energies["S"]:8.5f}'
        Cl_gap = exx_gaps["Cl"]["b"]
        Cl = f'{Cl_gap:8.5f} {exx_energies["Cl"]:8.5f}'
    else:
        Al_gap = ksinv_cahf_gaps["Al"][ref]["a"]
        if Al_gap is not None:
            Al = f'{Al_gap:8.5f} {ksinv_cahf_ref_en["Al"][ref]:8.5f}'
        else:
            Al = ""
        Si_gap = ksinv_cahf_gaps["Si"][ref]["a"]
        if Si_gap is not None:
            Si = f'{Si_gap:8.5f} {ksinv_cahf_ref_en["Si"][ref]:8.5f}'
        else:
            Si = ""
        S_gap = ksinv_cahf_gaps["S"][ref]["b"]
        if S_gap is not None:
            S = f'{S_gap:8.5f} {ksinv_cahf_ref_en["S"][ref]:8.5f}'
        else:
            S = ""
        Cl_gap = ksinv_cahf_gaps["Cl"][ref]["b"]
        if Cl_gap is not None:
            Cl = f'{Cl_gap:8.5f} {ksinv_cahf_ref_en["Cl"][ref]:8.5f}'
        else:
            Cl = ""
    print(f"{ref:6} {Al:20} {Si:20} {S:20} {Cl:20}")

               B                    C                     O                    F
EXX     0.00732 -24.52771    0.00945 -37.68648   -0.02818 -74.80872   -0.02882 -99.40831  
RS2    -0.01892 -24.64566    0.00279 -37.82847   -0.02682 -75.05395   -0.02869 -99.71810  
CISD   -0.01693 -24.65032   -0.01835 -37.83619   -0.03140 -75.05124   -0.03469 -99.71304  
AQCC   -0.01713 -24.65116   -0.01917 -37.83835   -0.03535 -75.05666   -0.03879 -99.71990  

               Al                    Si                     S                    Cl
EXX     0.00294 -241.87353   0.00392 -288.84948  -0.01280 -397.50055  -0.01253 -459.47712 
RS2    -0.00716 -242.31034   0.00058 -289.31684  -0.01443 -398.06517  -0.01526 -460.09718 
CISD   -0.00527 -242.30016  -0.00342 -289.30120  -0.01403 -398.03897  -0.01650 -460.07070 
AQCC   -0.00605 -242.31268  -0.00630 -289.31887  -0.02085 -398.06667  -0.02426 -460.10289 
